In [1]:
def process_strings(strings):
    """given lines of strings, cleans up and returns a set of non-whitespaced string"""
    stripped_strings = [s.strip() for s in strings]
    
    # Use a list to maintain the order and remove duplicates
    unique_strings = []
    
    for s in stripped_strings:
        if s not in unique_strings:
            unique_strings.append(s)
    
    # Sort the list of unique strings
    sorted_unique_strings = sorted(unique_strings)
    sorted_strings_with_new_line = ""
    for i in sorted_unique_strings:
        sorted_strings_with_new_line += f"{i}\n" 
    
    return sorted_strings_with_new_line

In [2]:
def get_table_and_col(raw_str: str):
    raw_str_list = raw_str.split(' ')
    # raw_str_list = [elem.split('\n') for elem in raw_str_list if '\n' in elem]
    tables_cols = [elem for elem in raw_str_list if '.' in elem]
    tables_cols = [f"""\"{elem.split('"')[1]}\": \"{elem.split('"')[-2]}\"""" for elem in  tables_cols]

    tables_cols = process_strings(tables_cols)

    return tables_cols

In [3]:
x = """WITH "CTE_Changes" AS (SELECT "CDPOS"."MANDANT",
                              "CDPOS"."TABKEY",
                              "CDHDR"."UDATE",
                              "CDHDR"."UTIME",
                              "CDHDR"."USERNAME",
                              ROW_NUMBER()
                              OVER (PARTITION BY "CDPOS"."TABKEY" ORDER BY "CDHDR"."UDATE", "CDHDR"."UTIME" desc) as "rn"
                       FROM "CDPOS" AS "CDPOS"
                                LEFT JOIN "CDHDR" AS "CDHDR"
                                          ON "CDHDR"."MANDANT" = "CDPOS"."MANDANT"
                                              AND "CDHDR"."CHANGENR" = "CDPOS"."CHANGENR"
                                              AND "CDHDR"."OBJECTCLAS" = "CDPOS"."OBJECTCLAS"
                                              AND "CDHDR"."OBJECTID" = "CDPOS"."OBJECTID"
                       WHERE "CDPOS"."OBJECTCLAS" = 'BANF'
                         AND "CDPOS"."TABNAME" = 'EBAN'
                         AND "CDPOS"."FNAME" = 'KEY'
                         AND "CDPOS"."CHNGIND" = 'I')
SELECT <%=sourceSystem%> || 'PurchaseRequisitionItem_' || "EBAN"."MANDT" || "EBAN"."BANFN" || "EBAN"."BNFPO" AS "ID",
       COALESCE(CAST("Changes"."UDATE" AS DATE) + CAST("Changes"."UTIME" AS TIME),
                CAST("EBAN"."BADAT" AS DATE) + CAST('00:00:01' AS TIME))                AS "CreationTime",
		 <%=sourceSystem%> || 'User_' || "EBAN"."MANDT" || "EBAN"."ERNAM"                                      AS "CreatedBy",
       Case
           WHEN "EBAN"."ESTKZ" = 'B' and "USR02"."USTYP" IS NULL THEN 'Automatic'
           WHEN "USR02"."USTYP" IN ('B', 'S', 'C') THEN 'Automatic'
           WHEN "USR02"."USTYP" = 'A' THEN 'Manual'
           ELSE NULL
           END                                                                          AS "CreationExecutionType",
       "EBAN"."LOEKZ"                                                                   AS "DeletionIndicator",
       "EBAN"."AFNAM"                                                                   AS "Requester",
		 <%=sourceSystem%> || 'Vendor_' || "EBAN"."MANDT" || "EBAN"."LIFNR"                                    AS "Vendor",
       "EBAN"."WAERS"                                                                   AS "Currency",
       "EBAN"."PREIS" / COALESCE("EBAN"."PEINH", 1)                                     AS "NetUnitPrice",
       "EBAN"."MENGE"                                                                   AS "Quantity",
       "EBAN"."MEINS"                                                                   AS "QuantityUnit",
       "EBAN"."STATU"                                                                   AS "ProcessingStatus",
		 <%=sourceSystem%> || 'ContractItem_' || "EBAN"."MANDT" || "EBAN"."KONNR" || "EBAN"."KTPNR"            AS "ContractItem",
		 <%=sourceSystem%> || 'Material_' || "EBAN"."MANDT" || "EBAN"."MATNR"                                  AS "Material",
		 <%=sourceSystem%> || 'Plant_' || "EBAN"."MANDT" || "EBAN"."WERKS"                                     AS "Plant",
       "EBAN"."FRGKZ"                                                                   AS "ReleaseIndicator",
       "T161U"."FKZTX"                                                                  AS "ReleaseIndicatorText",
       "EBAN"."TXZ01"                                                                   AS "ShortText",
       'SAP'                                                                            AS "SourceSystemType",
       "EBAN"."MANDT"                                                                   AS "SourceSystemInstance",
       "EBAN"."BANFN"                                                                   AS "SystemPurchaseRequisitionNumber",
       "EBAN"."BNFPO"                                                                   AS "SystemPurchaseRequisitionItemNumber"
FROM "EBAN" AS "EBAN"
         LEFT JOIN "CTE_Changes" AS "Changes"
                   ON "EBAN"."MANDT" = "Changes"."MANDANT"
                       AND "EBAN"."MANDT" || "EBAN"."BANFN" || "EBAN"."BNFPO" = "Changes"."TABKEY"
                       AND "Changes"."rn" = 1
         LEFT JOIN "T161U" AS "T161U"
                   ON "EBAN"."MANDT" = "T161U"."MANDT"
                       AND "EBAN"."FRGKZ" = "T161U"."FRGKZ"
                       AND "T161U"."SPRAS" = <%=LanguageKey%> 
         LEFT JOIN "USR02" AS "USR02"
                   ON "EBAN"."MANDT" = "USR02"."MANDT"
                       AND "EBAN"."ERNAM" = "USR02"."BNAME"
WHERE "EBAN"."MANDT" IS NOT NULL
  AND "EBAN"."BSTYP" = 'B'"""

In [4]:
y = get_table_and_col(x)

In [5]:
print(y)

"CDHDR": "CHANGENR"
"CDHDR": "MANDANT"
"CDHDR": "OBJECTCLAS"
"CDHDR": "OBJECTID"
"CDHDR": "UDATE"
"CDHDR": "USERNAME"
"CDHDR": "UTIME"
"CDPOS": "CHANGENR"
"CDPOS": "CHNGIND"
"CDPOS": "FNAME"
"CDPOS": "MANDANT"
"CDPOS": "OBJECTCLAS"
"CDPOS": "OBJECTID"
"CDPOS": "TABKEY"
"CDPOS": "TABNAME"
"Changes": "MANDANT"
"Changes": "TABKEY"
"Changes": "UDATE"
"Changes": "UTIME"
"Changes": "rn"
"EBAN": "AFNAM"
"EBAN": "BADAT"
"EBAN": "BANFN"
"EBAN": "BNFPO"
"EBAN": "BSTYP"
"EBAN": "ERNAM"
"EBAN": "ESTKZ"
"EBAN": "FRGKZ"
"EBAN": "KONNR"
"EBAN": "KTPNR"
"EBAN": "LIFNR"
"EBAN": "LOEKZ"
"EBAN": "MANDT"
"EBAN": "MATNR"
"EBAN": "MEINS"
"EBAN": "MENGE"
"EBAN": "PEINH"
"EBAN": "PREIS"
"EBAN": "STATU"
"EBAN": "TXZ01"
"EBAN": "WAERS"
"EBAN": "WERKS"
"T161U": "FKZTX"
"T161U": "FRGKZ"
"T161U": "MANDT"
"T161U": "SPRAS"
"USR02": "BNAME"
"USR02": "MANDT"
"USR02": "USTYP"

